In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv('data/train_large.csv')

In [ ]:
rep = [  
        "Facility Id",
        "CCS Procedure Description",
        "CCS Diagnosis Description",
        "APR DRG Description",
        "APR MDC Description",
        "APR Severity of Illness Description",
        "Unnamed: 0"
]
todrop = rep
# Features for polynomial
to_poly = ['Length of Stay','Birth Weight','Facility Name','APR DRG Code','CCS Diagnosis Code', 'CCS Procedure Code','Operating Certificate Number']
# Features for one hot encoding
to_onehot = [
        'APR Medical Surgical Description','APR Risk of Mortality', 'APR Severity of Illness Code','APR MDC Code',
        'Emergency Department Indicator','Health Service Area','Type of Admission',
        'Age Group', 'Gender', 'Race', 'Ethnicity','Emergency Department Indicator',
        'Payment Typology 1', 'Payment Typology 2', 'Payment Typology 3','Patient Disposition','Hospital County','Zip Code - 3 digits'
]

In [ ]:
data = data.drop(todrop,axis = 1)
X = data.drop(['Total Costs'],axis = 1)
y = data['Total Costs']

In [ ]:
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
ohe = OneHotEncoder(drop = 'first',sparse = False)
poly = PolynomialFeatures(degree = 2,include_bias = False)
X_cont = poly.fit_transform(X[to_poly])
X_ohe = ohe.fit_transform(X[to_onehot])
b = np.ones((X_cont.shape[0],1))
X = np.c_[b,X_cont,X_ohe]

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y,test_y = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LassoLars
model = LassoLars(alpha=0.1).fit(train_x,train_y)

In [ ]:
model.score(test_x,test_y),model.score(train_x,train_y),model.score(X,y)

In [ ]:
w = np.dot(np.linalg.inv(np.dot(train_x.T,train_x) + 10*np.identity(train_x.shape[1])),np.dot(train_x.T,train_y))

In [ ]:
from sklearn.metrics import r2_score
r2_score(train_y,np.dot(train_x,w)),r2_score(test_y,np.dot(test_x,w)),r2_score(y,np.dot(X,w))